## Load libraries

In [1]:
from gensim.models import KeyedVectors
import faiss
import numpy as np

In [2]:
NEIGHBORS = 50

word = 'проказа'

In [3]:
word_vectors_fpath = "models/wikipedia-ru-2018.word_vectors"

In [4]:
wv = KeyedVectors.load_word2vec_format(word_vectors_fpath, binary=False, unicode_errors="ignore")

/home/denis/Programs/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [5]:
wv.init_sims(replace=True)

## knn by w2v

In [6]:
def get_nns(target, topn=NEIGHBORS):
    """
    target - word token
    topn - how many neighbors
    """
    nns = wv.most_similar(positive=[target], negative=[], topn=topn)
    #nns = [(word, score) for word, score in nns if minimize(word) != minimize(target)]
    return nns

In [7]:
%%time
get_nns(word)

CPU times: user 258 ms, sys: 12 ms, total: 270 ms
Wall time: 384 ms


[('малярия', 0.7535935640335083),
 ('гонорея', 0.7530273199081421),
 ('брюшной_тиф', 0.7527637481689453),
 ('хроническая', 0.7498410940170288),
 ('энцефалопатия', 0.7485659122467041),
 ('чесотка', 0.7483586072921753),
 ('волчанка', 0.7467740774154663),
 ('сибирская_язва', 0.744862973690033),
 ('пневмония', 0.7420924305915833),
 ('экзема', 0.7396251559257507),
 ('акатизия', 0.7387583255767822),
 ('диарея', 0.7369372844696045),
 ('панкреатит', 0.7364274263381958),
 ('ревматоидный_артрит', 0.7353441715240479),
 ('идиопатическая', 0.7351337671279907),
 ('кожный_зуд', 0.7348761558532715),
 ('скарлатина', 0.7332731485366821),
 ('энтерит', 0.7330397963523865),
 ('нейродермит', 0.7294857501983643),
 ('сифилис', 0.7289275527000427),
 ('корь', 0.7280865907669067),
 ('простатит', 0.7264933586120605),
 ('вирусная_инфекция', 0.7261444926261902),
 ('конъюнктивит', 0.7260156869888306),
 ('гангрена', 0.7253180742263794),
 ('себорея', 0.7248758673667908),
 ('склеродермия', 0.7244752645492554),
 ('сарко

## KNN by faiss

In [8]:
index = faiss.IndexFlatIP(wv.vector_size)
index.add(wv.syn0norm)

/home/denis/Programs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.vectors_norm instead).
  


In [9]:
def get_nns_faiss(target, topn=NEIGHBORS):
    """
    Get target neighbors by faiss
    """
    nns_list = list()

    numpy_vec = np.array([wv[target]])
    D, I = index.search(numpy_vec, topn + 1)

    for _D, _I in zip(D, I):
        for n, (d, i) in enumerate(zip(_D.ravel(), _I.ravel())):
            if n > 0:
                nns_list.append((wv.index2word[i], d))

    return nns_list

In [10]:
%%time
get_nns_faiss(word)

CPU times: user 1.26 s, sys: 44.2 ms, total: 1.31 s
Wall time: 339 ms


[('малярия', 0.75359356),
 ('гонорея', 0.75302726),
 ('брюшной_тиф', 0.75276375),
 ('хроническая', 0.74984103),
 ('энцефалопатия', 0.74856585),
 ('чесотка', 0.7483586),
 ('волчанка', 0.74677414),
 ('сибирская_язва', 0.74486303),
 ('пневмония', 0.7420925),
 ('экзема', 0.73962516),
 ('акатизия', 0.7387583),
 ('диарея', 0.73693734),
 ('панкреатит', 0.7364274),
 ('ревматоидный_артрит', 0.7353442),
 ('идиопатическая', 0.73513377),
 ('кожный_зуд', 0.73487616),
 ('скарлатина', 0.7332732),
 ('энтерит', 0.7330398),
 ('нейродермит', 0.72948575),
 ('сифилис', 0.72892755),
 ('корь', 0.72808653),
 ('простатит', 0.7264933),
 ('вирусная_инфекция', 0.7261445),
 ('конъюнктивит', 0.7260157),
 ('гангрена', 0.7253181),
 ('себорея', 0.72487587),
 ('склеродермия', 0.72447526),
 ('саркома', 0.7239408),
 ('кишечная_непроходимость', 0.7237814),
 ('псориаз', 0.7226931),
 ('эпилепсия', 0.72225845),
 ('стенокардия', 0.72158694),
 ('радикулит', 0.72133183),
 ('желтуха', 0.72089016),
 ('подагра', 0.7205219),
 ('сис